### Downloading Images

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!conda env create -f airnet.yaml

/bin/bash: line 1: conda: command not found


In [ ]:
!mkdir -p dataset/images
!wget https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip \
      https://vizwiz.cs.colorado.edu/VizWiz_final/images/val.zip \
      https://vizwiz.cs.colorado.edu/VizWiz_final/images/test.zip

--2024-04-26 15:37:44--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip
Resolving vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)... 198.59.7.50
Connecting to vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)|198.59.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11298421598 (11G) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  10.52G   195MB/s    in 63s     

2024-04-26 15:38:47 (171 MB/s) - ‘train.zip’ saved [11298421598/11298421598]

--2024-04-26 15:38:47--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/val.zip
Reusing existing connection to vizwiz.cs.colorado.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: 3488913457 (3.2G) [application/zip]
Saving to: ‘val.zip’

val.zip             100%[===================>]   3.25G   203MB/s    in 19s     

2024-04-26 15:39:06 (178 MB/s) - ‘val.zip’ saved [3488913457/3488913457]

--2024-04-26 15:39:06--  https://vizwiz.cs.colorado.edu/VizWiz_

In [ ]:
!unzip -q -o train.zip -d dataset/images
!unzip -q -o val.zip -d dataset/images
!unzip -q -o test.zip -d dataset/images

In [ ]:
!rm train.zip val.zip test.zip

In [14]:
!pip3 install virtualenv
!virtualenv "/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/virtual_env"
!chmod 755 "/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/virtual_env/bin/activate"; pip3 install timm
!pip3 install openmim
!mim install mmcv
!pip install gdown

created virtual environment CPython3.10.12.final.0-64 in 9597ms
  creator CPython3Posix(dest=/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/virtual_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.5.1, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.2.0/index.html
ERROR: Operation cancelled by user
^C


### Airnet

In [5]:
!git clone https://github.com/XLearning-SCU/2022-CVPR-AirNet

Cloning into '2022-CVPR-AirNet'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 124 (delta 4), reused 3 (delta 3), pack-reused 112
Receiving objects: 100% (124/124), 36.54 MiB | 24.31 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [7]:
import os
import argparse
import json
from datetime import datetime

import numpy as np
import shutil
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms

import sys
# add the path of the virtual environmentsite-packages to colab system path
sys.path.append("virtual_env/lib/python3.10/site-packages")
import timm

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
    ])

In [ ]:
####################################
# I HAVE NO IDEA IF THIS WORKS     #
# THIS IS SOME RANDOM CHATGPT CODE #
####################################
!mkdir airnet_data
!mkdir transformed_images/images

# Praise ChatGPT
input_dir = 'dataset/images/'
output_dir = 'transformed_images/images'

shutil.rmtree(output_dir, ignore_errors=True)
shutil.copytree(input_dir, output_dir)

for root, dirs, files in os.walk(output_dir):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Open the image
            img = Image.open(os.path.join(root, file))

            # Apply the transformation pipeline
            transformed_img = test_transform(img)

            # Save the transformed image, preserving the folder structure
            output_subdir = root.replace(output_dir, input_dir)
            os.makedirs(output_subdir, exist_ok=True)
            transformed_img.save(os.path.join(output_subdir, file))

In [15]:
!mkdir ckpt
import gdown
url = 'https://drive.google.com/file/d/1qh5F6_u4NWnMMlw2q-mNzc4-7y427dsX/view'
output = 'ckpt/All.pth'
gdown.download(url, output, quiet=False)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1qh5F6_u4NWnMMlw2q-mNzc4-7y427dsX
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1qh5F6_u4NWnMMlw2q-mNzc4-7y427dsX/view?usp=drive_link
To: /content/ckpt/All.pth
82.9kB [00:00, 22.5MB/s]


'ckpt/All.pth'

In [ ]:
!python 2022-CVPR-AirNet/demo.py --test_path='./dataset/images/train/' --output_path='./airnet_data/' --mode 3

Start testing...
  0% 12/23954 [02:53<70:10:22, 10.55s/it]